# Grouping

We saw in our cursory review of the descriptives that there is evidence the data is skewed. We also saw that there are some other interesting patterns to the data. We saw for example that the range of scores was quite large. However, we also saw that there appeared to a large amount of intra-statistic-consistency across each of the areas measured (Open, High, Low etc.). 

Ahead of further analysis we want to understnd these patterns better, such that we better frame further tests. It might be that there are companies we want to exclude, it may be that there are specific time period that impact the data. These findings might tell us we need to transform the data for tests we will later use. We may find that the year has an impact on the data also, which will further change how we want to analyse further.

Importantly, there are 500 companies in the data with 9 columns of values daily over a 5 year period. This is equivalent to approximately **5,850,000** individual values. This poses a significant computational challenge. We therefore need to find a way to minimize this number so that we can more effectively compute. We will therefore average high and low scores for each company and use only these means. 

We will then Find the first and fourth quartiles of this data. We can then compare these score and more importantly, test to see if either the fourth or the 1st quartile differ significntly enough from the second and third quartiles.

**We will do the following:**
- Find the top quartile of companies
- Find the bottom quartile of companies
- Compare first and fourth with the second and third
- Compare descriptives for groups based on company
- Compare descriptives for years
- Test skewness and goodness of fit directly

In [1]:
# Import modules and variables we need 
import yfin_descriptives_py # import our previous work
from yfin_descriptives_py import x_1, x_2, x_3, x_4, x_5, x_6, x_7 # import arrays
from yfin_descriptives_py import di_mean1, di_median1, di_max_min_range, di_var1, di_std1 # import disctionaries
import numpy as np
import pandas as pd

                        Date        Open        High         Low       Close  \
0  2018-11-29 00:00:00-05:00   43.829761   43.863354   42.639594   43.083508   
1  2018-11-29 00:00:00-05:00  104.769074  105.519257  103.534595  104.636131   
2  2018-11-29 00:00:00-05:00   54.176498   55.007500   54.099998   54.729000   
3  2018-11-29 00:00:00-05:00   83.749496   84.499496   82.616501   83.678497   
4  2018-11-29 00:00:00-05:00   39.692784   40.064904   38.735195   39.037853   

      Volume  Dividends  Stock Splits Company  
0  167080000       0.00           0.0    AAPL  
1   28123200       0.00           0.0    MSFT  
2   31004000       0.00           0.0   GOOGL  
3  132264000       0.00           0.0    AMZN  
4   54917200       0.04           0.0    NVDA  


In [ ]:
# Finding the fourth quartile of data

## 